# EDA KAGGLE VS REAL TRANSACTIONS

## IMPORTS

In [1]:
import findspark
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, plot
import seaborn as sns
from ks_crypto.lib.spark_conn import create_yarn_connection
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

pd.set_option('display.max_rows', 500)

init_notebook_mode(connected=True)
findspark.init()
spark = create_yarn_connection()

/opt/conda/default/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
DANON_FULL_TABLENAME = "kschool-crypto:ks_crypto_dataset.danon_transactions"
TRANSACTIONS_FULL_TABLENAME = "bigquery-public-data:crypto_bitcoin.transactions"

## 1. READ DATA

In [3]:
danon_df = \
    spark.read.format('bigquery') \
    .option('table', DANON_FULL_TABLENAME) \
    .load() \
    .select('transaction_hash')\
    .persist()

danon_df.count()

202804

In [4]:
transactions_df = \
    spark.read.format('bigquery') \
    .option('table', TRANSACTIONS_FULL_TABLENAME) \
    .load()\
    .select(F.col('hash').alias('transaction_hash'),
            'lock_time',
            'block_hash',
            'block_timestamp',
            'block_timestamp_month',
            'input_count',
            'output_count',
            'input_value',
            'output_value',
            'is_coinbase',
            'fee')

## 2. JOIN REAL TRANSACTIONS WITH KAGGLE ONES

In [ ]:
sampled_transactions_df = \
    transactions_df\
    .join(danon_df,
          on=['transaction_hash'],
          how='inner')\
    .persist()

sampled_transactions_df.count()

## 3. OBTAIN MIN-MAX DATES OF TRANSACTIONS

In [ ]:
sampled_transactions_df.select(F.min('block_timestamp_month'), F.max('block_timestamp_month')).show()

In [ ]:
transactions_df.filter(F.col('block_timestamp_month').between('2016-01-01', '2017-10-01')).count()

In [ ]:
sampled_transactions_df.filter(F.col('block_timestamp_month').between('2016-01-01', '2017-10-01')).count()

## 4. CHECK HOW MANY DIF DATES

In [ ]:
period_transactions_df = \
    transactions_df.filter(F.col('block_timestamp_month').between('2016-01-01', '2017-10-01'))\
    .persist()

period_transactions_df.count()

In [ ]:
sampled_transactions_df.count()

In [ ]:
sampled_transactions_df\
    .drop_duplicates(['block_timestamp'])\
    .count()

In [ ]:
ts = \
    sampled_transactions_df\
    .groupBy('block_timestamp')\
    .count()\
    .orderBy('block_timestamp')\
    .toPandas()

px.line(ts, x='block_timestamp', y='count')

In [ ]:
transactions_df\
    .filter((F.col('block_timestamp_month')== '2016-01-01') & (F.col('block_timestamp') == '2016-01-13 23:40:57'))\
    .count()

## 5. CHECK DIFF HOURS IN THE FIRST AND LAST EVENT

In [ ]:
sampled_transactions_df\
    .select('block_timestamp')\
    .withColumn('lag_block_timestamp', F.lag('block_timestamp').over(w))\
    .withColumn('diff_hours', (F.col('block_timestamp').cast('long') -  F.col('lag_block_timestamp').cast('long')) / 3600)\
    .show()

In [ ]:
w_ord = Window.orderBy('block_timestamp')
w_id = Window.partitionBy('block_timestamp')

diff_hours_lag_fun = (F.col('block_timestamp').cast('long') -  F.col('lag_block_timestamp').cast('long')) / 3600
is_more_hours_lag_cond = (diff_hours_lag_fun > 3) | (F.col('lag_block_timestamp').isNull())

diff_hours_lead_fun = (F.col('lead_block_timestamp').cast('long') -  F.col('block_timestamp').cast('long')) / 3600
is_more_hours_lead_cond = (diff_hours_lead_fun > 3) | (F.col('lead_block_timestamp').isNull())


first_last_transactions_df = \
sampled_transactions_df\
    .select('block_timestamp')\
    .withColumn('lag_block_timestamp', F.lag('block_timestamp').over(w_ord))\
    .withColumn('is_first_in_period', F.when(is_more_hours_lag_cond, 1).otherwise(0))\
    .withColumn('is_first_in_period', F.max('is_first_in_period').over(w_id))\
    .withColumn('lead_block_timestamp', F.lead('block_timestamp').over(w_ord))\
    .withColumn('is_last_in_period', F.when(is_more_hours_lead_cond, 1).otherwise(0))\
    .withColumn('is_last_in_period', F.max('is_last_in_period').over(w_id))\
    .drop(*['lead_block_timestamp', 'lag_block_timestamp'])

In [ ]:
first_last_transactions_df\
    .filter((F.col('is_last_in_period') == 1) | (F.col('is_first_in_period') == 1))\
    .dropDuplicates()\
    .withColumn('lead_block_timestamp', F.lead('block_timestamp').over(w_ord))\
    .withColumn('range_hours', diff_hours_lead_fun)\
    .filter((F.col('is_first_in_period') == 1))\
    .groupBy('range_hours').count().orderBy(F.desc('range_hours'))\
    .show()

In [ ]:
first_last_transactions_df\
    .groupBy('block_timestamp')\
    .agg(F.count(F.lit(1)).alias('count'), 
         F.sum('is_first_in_period').alias('sum_is_first_in_period'),
         F.sum('is_last_in_period').alias('sum_is_last_in_period'))\
    .show()

## 6. PLOT GROUPED VARS 

In [86]:
num_cols = ['input_count', 'output_count', 'input_value', 'output_value']
mean_fun_list = [F.mean(c).alias('mean_'+ c) for c in num_cols]
sum_fun_list = [F.sum(c).alias('sum_'+ c) for c in num_cols]
median_fun_list = [F.expr(f'percentile_approx({c}, 0.5)').alias('median_'+c) for c in num_cols]

full_ts = \
    period_transactions_df\
    .groupBy(F.date_trunc('hour', 'block_timestamp').alias('block_timestamp'))\
    .agg(F.count(F.lit(1)).alias('count'),
         *mean_fun_list,
         *sum_fun_list,
         *median_fun_list)\
    .orderBy('block_timestamp')\
    .toPandas()

full_ts.count()

block_timestamp        16047
count                  16047
mean_input_count       16047
mean_output_count      16047
mean_input_value       16047
mean_output_value      16047
sum_input_count        16047
sum_output_count       16047
sum_input_value        16047
sum_output_value       16047
median_input_count     16047
median_output_count    16047
median_input_value     16047
median_output_value    16047
dtype: int64

In [87]:
num_cols = ['input_count', 'output_count', 'input_value', 'output_value']
mean_fun_list = [F.mean(c).alias('mean_'+ c) for c in num_cols]
sum_fun_list = [F.sum(c).alias('sum_'+ c) for c in num_cols]
median_fun_list = [F.expr(f'percentile_approx({c}, 0.5)').alias('median_'+c) for c in num_cols]

ts = \
sampled_transactions_df\
    .groupBy(F.date_trunc('hour', 'block_timestamp').alias('block_timestamp'))\
    .agg(F.count(F.lit(1)).alias('count'),
         *mean_fun_list,
         *sum_fun_list,
         *median_fun_list)\
    .orderBy('block_timestamp')\
    .toPandas()
ts.count()

block_timestamp        121
count                  121
mean_input_count       121
mean_output_count      121
mean_input_value       121
mean_output_value      121
sum_input_count        121
sum_output_count       121
sum_input_value        121
sum_output_value       121
median_input_count     121
median_output_count    121
median_input_value     121
median_output_value    121
dtype: int64

In [90]:
v_list = ['count', 'mean_input_count', 'mean_output_count',
       'mean_input_value', 'mean_output_value', 'sum_input_count',
       'sum_output_count', 'sum_input_value', 'sum_output_value',
       'median_input_count', 'median_output_count', 'median_input_value',
       'median_output_value']

for var in v_list:

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=full_ts['block_timestamp'], y=full_ts[var],
                             mode='lines',
                             name='full_ts'))
    fig.add_trace(go.Scatter(x=ts['block_timestamp'], y=full_ts[var],
                             mode='lines+markers',
                             name='ts'))

    plot(fig, filename = f'{var}.html', auto_open=False)